# Training LuNA 16 Dataset

**The basic structure of what we’re going to implement is as follows:**

- Initialize our model and data loading.
- Loop over a semi-arbitrarily chosen number of epochs.
  - Loop over each batch of training data returned by dataset.
    - The data-loader worker process loads the relevant batch of data in the background.
    - Pass the batch into the model to get predictions.
    - Calculate our loss based on the difference between our predicted results and our ground-truth data.
    - Record metrics about our model’s performance into a temporary data structure.
    - Update the model weights via backpropagation of the error.
  - Loop over each batch of validation data (in a manner very similar to the training loop).
    - Load the relevant batch of validation data (again, in the background worker process).
    - Classify the batch, and compute the loss.
    - Record information about how well the model performed on the validation data.
  - Log progress and performance information for this epoch.


In [ ]:
from torch.utils import data as data_utils

from luna16 import augmentations, datasets, dto

# Loading LuNA Data

Training on single samples at a time is typically an inefficient use of computing resources, because most processing platforms are capable of more parallel calculations than are required by a model to process a single training or validation sample. The solution is to group sample tuples together into a batch tuple, allowing multiple samples to be processed at the same time.

Conveniently, we don’t have to implement any of this _batching_: the PyTorch `DataLoader` class will handle all of the collation work for us. We’ve already built the bridge from the CT scans to PyTorch tensors with our `CutoutsDataset` class, so all that remains is to plug our dataset into a data loader.


In [ ]:
ratio = dto.NoduleRatio(positive=1, negative=5)
transformations: list[augmentations.Transformation] = [
    augmentations.Flip(),
    augmentations.Offset(offset=0.1),
    augmentations.Scale(scale=0.2),
    augmentations.Rotate(),
]
filters: list[augmentations.Filter] = [
    augmentations.Noise(noise=25.0),
]
dataset = datasets.CutoutsDataset(
    ratio=ratio,
    validation_stride=20,
    transformations=transformations,
    filters=filters,
)

train_dataloader = data_utils.DataLoader(
    dataset=dataset,
    batch_size=64,
    num_workers=0,
    pin_memory=False,
)

## Data Module

The `DataModule` class is a custom data handling module built on top of PyTorch's `DataLoader`. It simplifies the process of creating and managing data loaders for training and validation datasets. The `DataModule` class handles the following tasks:

- Initializes with training and validation datasets, along with a specified batch size.
- Determines the device (CPU or GPU) to be used for data loading and model training.
- Creates data loaders for both training and validation datasets, ensuring efficient data loading with support for multiple workers and CUDA pin memory if available.
- Provides methods to retrieve the appropriate data loader (training or validation) based on the context.

This abstraction helps in organizing the data loading logic and makes the training and validation loops cleaner and more manageable.

In [ ]:
data_module = datasets.DataModule(
    batch_size=64,
    train=dataset,
    validation=dataset,  # this would normally be a validation dataset
)

# LuNA Model Architecture

Classification models often have a structure that consists of a tail, a backbone (or body), and a head. 

Using a structure like this can be a good first building block for a convolutional network. There are more complicated designs out there, but for many projects they’re overkill in terms of both implementation complexity and computational demands. It’s a good idea to start simple and add complexity only when there’s a demonstrable need for it.


## Initialization

Kaiming Normal initialization was chosen for weight initialization because it's designed to work well with the ReLU activation function. This approach helps maintain a healthy gradient flow through the network, preventing vanishing or exploding gradients even in deep architectures. 

Biases were initialized to zero to start with no additional offset, allowing the network to rely solely on the learned weights to adjust activation outputs.

We are using `fan_in` mode because it is default and it seems to be good place to start.


## Network Composition

Networks are typically composed of three parts:

- **Tail**
- **Backbone**
- **Head**

These parts are described in more detail below.


### Tail

The _tail_ is the first few layers that process the input to the network. These early layers often have a different structure or organization than the rest of the network, as they must adapt the input to the form expected by the backbone. 

#### Our implementation

We are going to normalize our input using `nn.BatchNorm3d`, that will shift and scale our input so that it has a mean of 0 and a standard deviation of 1. Thus, the somewhat odd Hounsfield unit (HU) scale that our input is in won’t really be visible to the rest of the network.


### Backbone (or body)

The _backbone_ of the network typically contains the bulk of the layers, which are usually arranged in series of blocks. Each block has the same (or at least a similar) set of layers, though often the size of the expected input and the number of filters changes from block to block.

#### Our implementation

Our backbone consists of variable number of blocks, with the block implementation pulled out into the separate `nn.Module` subclass. 

Since each block ends with a `2 × 2 × 2` max-pool operation, after for example 4 layers we will have decreased the resolution of the image 16 times in each dimension. In this case, our data is returned in chunks that are `32 × 48 × 48` (`cutout_shape` in code), which will become `2 × 3 × 3` by the end of the backbone.


### Head

The _head_ of the network takes the output from the backbone and converts it into the desired output form. 

For convolutional networks, this often involves flattening the intermediate output and passing it to a fully connected layer.

#### Our implementation

Before we pass data into a fully connected layer, we must flatten it using the `flatten` function. Since that operation is stateless (it has no parameters that govern its behavior), we can simply perform the operation in the forward function.

Our head is just a fully connected layer followed by a call to `nn.Softmax`. Softmax is a useful function for single-label classification tasks and has a few nice properties: it bounds the output between 0 and 1, it’s relatively insensitive to the absolute range of the inputs (only the relative values of the inputs matter), and it allows our model to express the degree of certainty it has in an answer.

For the return value of the forward method, we return both the raw _logits_ and the softmax-produced _probabilities_.

We’ll use the logits when we calculate the `nn.CrossEntropyLoss` during training, and we’ll use the probabilities for when we want to actually classify the samples. This kind of slight difference between what’s used for training and what’s used in production is fairly common, especially when the difference between the two outputs is a simple, stateless function like softmax.

> **Note**: _Logits_ are the numerical values produced by the network prior to being normalized into probabilities by the softmax layer. Logits are really just the raw input to the softmax layer. They can have any real-valued input, and the softmax will squash them to the range 0–1.


## Model Block

This block will be repeated multiple times to form our model’s backbone. This is the part of the network where we will be experimenting the most so components mentioned below are subject to change and not every model will have all of them.


### Convolutional Layers

We’re using `3 × 3 × 3` convolutions in our block. A single `3 × 3 × 3` convolution has a receptive field of `3 × 3 × 3`, which is almost tautological. Twenty-seven voxels are fed in, and one comes out. 

It gets interesting when we use two `3 × 3 × 3` convolutions stacked back to back. Stacking convolutional layers allows the final output voxel (or pixel) to be influenced by an input further away than the size of the convolutional kernel suggests. If that output voxel is fed into another `3 × 3 × 3` kernel as one of the edge voxels, then some of the inputs to the first layer will be outside of the `3 × 3 × 3` area of input to the second. The final output of those two stacked layers has an effective receptive field of `5 × 5 × 5`. That means that when taken together, the stacked layers act as similar to a single convolutional layer with a larger size.

#### Biased vs. Unbiased Convolution

The `bias` parameter in a convolutional layer is a learnable offset that is added to the output of the convolution. It’s a way to allow the network to adjust the output of the convolutional layer in a way that isn’t dependent on the input.

When using batch normalization, we explicitly set `bias` to `False. Batch normalizations already includes a learnable bias term. Having bias in both convolutional layer and batch normalization is redundant since the batch normalization bias can accomplish the same thing. 


### MaxPool

The output of our two stacked convolutions is fed into a `2 × 2 × 2` max pool, which means we’re taking a `6 × 6 × 6` effective field, throwing away seven-eighths of the data, and going with the one `5 × 5 × 5` field that produced the largest value. Now, those “discarded” input voxels still have a chance to contribute, since the max pool that’s one output voxel over has an overlapping input field, so it’s possible they’ll influence the final output that way.


### ReLU

In the `nn.ReLU` loss functions, the outputs greater than $0.0$ will be left unchanged, and outputs less than $0.0$ will be clamped to zero.


### Batch Normalization

`BatchNorm3d` is a PyTorch module that applies batch normalization specifically designed for 3D inputs. It normalizes each channel of a 5D tensor (with shape [N, C, D, H, W]), ensuring that the outputs have a standard distribution (typically zero mean and unit variance) during training. 

This normalization helps reduce internal covariate shift, which can lead to faster convergence and improved overall stability of the network during training. Additionally, `BatchNorm3d` can provide a regularizing effect, potentially reducing the need for other forms of regularization like dropout.

> **Note**: [N, C, D, H, W] stands for:
> - **N** = **Batch size** (Number of samples)
> - **C** = **Channels** (like RGB for images, or feature channels)
> - **D** = **Depth** (the z-axis)
> - **H** = **Height** (the y-axis)
> - **W** = **Width** (the x-axis)


### Dropout

Dropout is a regularization technique used to prevent overfitting in neural networks. During training, dropout randomly sets a fraction of the input units to zero at each update, which helps the model to not rely too much on any individual neuron. As a result, it encourages the network to learn more robust, distributed features.

#### Dropout vs. Dropout3d

- **Dropout:** Applies dropout independently to each element (neuron) in a layer, which works well for fully connected layers or 2D convolutional layers. It randomly drops out neurons, forcing the network to rely on a combination of features rather than any single path.
  
- **Dropout3d:** Specifically designed for 3D data (commonly used in 3D convolutional neural networks), Dropout3d drops whole channels (i.e., entire feature maps) instead of individual neurons. This helps preserve the spatial structure of the data and avoids artifacts that can occur when individual voxels in a feature map are dropped at random.

Sometimes, even when working with 3D data, standard Dropout may be preferred over Dropout3d. In scenarios where the model benefits from preserving complete feature maps across channels, yet still requires regularization at a more granular level, using Dropout can allow selective deactivation of individual elements. This can be particularly effective when each voxel carries critical localized information that might be lost if entire channels were dropped.
